In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
# import zipfile
# with zipfile.ZipFile("/kaggle/input/instacart-market-basket-analysis/order_products__prior.csv.zip","r") as zipf:
# zipf.extractall(".")

In [ ]:
#  Pick a Dataset you might be interested in.
#  Say, all airline-safety files...
prior = "order_products__prior.csv"
train = "order_products__train.csv"
orders = "orders.csv"
products = "products.csv"
aisles = "aisles.csv"
depart = "departments.csv"

In [ ]:
import zipfile
# Will unzip the files so that you can see them..
with zipfile.ZipFile("/kaggle/input/instacart-market-basket-analysis/"+prior+".zip","r") as z:
    z.extractall(".")
    
from subprocess import check_output
print(check_output(["ls", prior]).decode("utf8"))

from pandas import DataFrame as df
prior = pd.read_csv("order_products__prior.csv")
prior.head()

In [ ]:
with zipfile.ZipFile("/kaggle/input/instacart-market-basket-analysis/"+train+".zip","r") as z:
    z.extractall(".")
train = pd.read_csv("order_products__train.csv")

In [ ]:
with zipfile.ZipFile("/kaggle/input/instacart-market-basket-analysis/"+orders+".zip","r") as z:
    z.extractall(".")
orders = pd.read_csv("orders.csv")

In [ ]:
with zipfile.ZipFile("/kaggle/input/instacart-market-basket-analysis/"+products+".zip","r") as z:
    z.extractall(".")
products = pd.read_csv("products.csv")

In [ ]:
with zipfile.ZipFile("/kaggle/input/instacart-market-basket-analysis/"+aisles+".zip","r") as z:
    z.extractall(".")
aisles = pd.read_csv("aisles.csv")

In [ ]:
with zipfile.ZipFile("/kaggle/input/instacart-market-basket-analysis/"+depart+".zip","r") as z:
    z.extractall(".")
depart = pd.read_csv("departments.csv")

**aisles : 1~133 제품번호 및 제품명<br>**
**departments : 1~21 제품 카테고리<br>
orders : order_id / user_id	/eval_set /order_number /order_dow /order_hour_of_day /days_since_prior_order 3421083 rows × 7 columns<br>**
order_id : 1~206209<br> user_id : 1~206209<br> eval_set은 prior/train으로 어디에 분류된 주문인지 적혀있음.<br> order_number는 한 주문자의 주문상품들에 대해 순서대로 1,2,3,,,적어둔것.<br> order_dow는 모르겠음.<br> order_hour는 주문한 시간.즉,전부 다 한번에 주문한건 아님<br>days_since..는 이전 주문부터 다음 주문까지 걸린 날짜. 실수값이고 처음부터 NaN이 보임.<br>
**products :product_id/ product_name /aisle_id /department_id 49688 rows × 4 columns<br>**
제품 하나하나의 정확한 상품명. 분류가 products < aisle < department 인듯<br>
aisle이 통로인데 그럼 매장 내에서의 섹션을 말하나..?<br>
<br>
<br>
**order_products__prior : order_id / product_id / add_to_cart_order / reordered 순으로 된 파일 32434489 rows × 4 columns<br>
order_products__train : order_id / product_id / add_to_cart_order / reordered 순으로 된 파일 1384617 rows × 4 columns<br>**
order_id순으로 분류되어있고(serialx) 각각에 대한 제품id, 카트에 담은 순서, 재주문인지 아닌지의 라벨<br>
여기서의 order_id는 orders의 order_id랑 동일함을 아래에서 확인함.<br>
reordered --> 0,1로 구성된 정답레이블로 추정. 재구매유무를 찾는 대회니까.

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
np.max(orders['user_id']) 
#order_id from orders = 3421083
#order_id from train.csv= 3421070
#order_id from prior.csv = 3421083

In [ ]:
dow=list(set(orders['order_dow']))
print(dow)
# dow : 요일. 0 - 일요일

In [ ]:
oid_prior=list(set(prior['order_id']))
print(len(oid_prior))
oid_train=list(set(train['order_id']))
print(len(oid_train))
oid=list(set(orders['order_id']))
print(len(oid))

# 131209 + 3214874 =/= 3421083 (75,000차이) -> sample_submission 행 수와 일치 : test set

In [ ]:
len(orders)

In [ ]:
print(100*((len(oid)-len(oid_train)-len(oid_prior))/len(oid)),'%')

In [ ]:
for col in orders.columns:
    null_or = 'column: {:>10}\t Percent of NaN value: {:.2f}%'.format(col, 100* (orders[col].isnull().sum() / orders[col].shape[0]))
    print(null_or)
    
    # orders['days_since_prior_order']에만 6.03% NaN

In [ ]:
for col in products.columns:
    null = 'column: {:>10}\t Percent of NaN value: {:.2f}%'.format(col, 100 * (products[col].isnull().sum() / products[col].shape[0]))
    print(null)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('seaborn')
sns.set(font_scale=2.5) 
import missingno as msno

#ignore warnings
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

msno.matrix(df=orders.iloc[:, :], figsize=(6, 6), color=(0.8, 0.5, 0.2))

In [ ]:
msno.bar(df=orders.iloc[:, :], figsize=(6, 6), color=(0.8, 0.5, 0.2))

In [ ]:
f, ax = plt.subplots(1, 2, figsize=(15, 5))

prior['reordered'].value_counts().plot.pie(explode=[0, 0.1], autopct='%1.1f%%', ax=ax[0], shadow=True)
ax[0].set_title('Pie plot - reordered')
ax[0].set_ylabel('')
sns.countplot('reordered', data=prior, ax=ax[1])
ax[1].set_title('Count plot - reordered')

plt.show()

In [ ]:
f, ax = plt.subplots(1, 2, figsize=(10, 5))

train['reordered'].value_counts().plot.pie(explode=[0, 0.1], autopct='%1.1f%%', ax=ax[0], shadow=True)
ax[0].set_title('Pie plot - reordered')
ax[0].set_ylabel('')
sns.countplot('reordered', data=prior, ax=ax[1])
ax[1].set_title('Count plot - reordered')

plt.show()

In [ ]:
print(len(products))

In [ ]:
# sub = pd.read_csv('/kaggle/input/instacart-market-basket-analysis/sample_submission.csv')
# sub.head()

sub = "sample_submission.csv"
with zipfile.ZipFile("/kaggle/input/instacart-market-basket-analysis/"+sub+".zip","r") as z:
    z.extractall(".")
sub = pd.read_csv("sample_submission.csv")
sub.head()

In [ ]:
o2 = orders.set_index("user_id")
o2.head()

**user_id로 묶어 정보가 검색 가능한 dictionary 만들기**

In [ ]:
o_dict = dict(list(orders[:][['order_id', 'eval_set', 'user_id' , 'order_number', 'order_dow' , 'order_hour_of_day' , 'days_since_prior_order']].groupby('user_id')))

In [ ]:
o_dict[1]

In [ ]:
o_num = list(orders.groupby('user_id').size())
# print(o_num)
np.max(o_num)

order number의 갯수를 counter로 세서<br>
주문횟수(order number, o_num)를 key로 갖고 key값만큼 주문한 적 있는 고객의 수를 value로 갖는 dictionary 생성
::<br>
o_num : list<br>
result : dictionary

In [ ]:
from collections import Counter

result = Counter(o_num)
print(result)

In [ ]:
for k in result.keys():
    print(k,result[k])
print()

In [ ]:
se_result = pd.Series(result, name='count')

In [ ]:
se_result = se_result.sort_values(ascending=False)

In [ ]:
se_result.index.name='o_num'
# se_result.reset_index()

In [ ]:
df_result = pd.DataFrame(se_result)
df_result.reset_index(level=['o_num'], inplace = True)
df_result = df_result.sort_values(by=['o_num'], ascending=True)
df_result.tail()

In [ ]:
## from matplotlib import pyplot as plt
plt.style.use('seaborn')
sns.set(font_scale=2.5)

%matplotlib inline  
 
# y = se_result[:]
# x = range(len(y))
# plt.bar(x, y, width = 0.8, color="blue")
# plt.show()

In [ ]:
sns.set(font_scale=0.9)
plt.figure(figsize=(30,10)) # 위치가 sns.plot 위에 있어야만 작동하나..?

ax = sns.barplot(data=df_result, x="o_num", y="count")
ax.set_xticklabels(ax.get_xticklabels(), rotation=0, ha="right")
plt.title("customer number per order_num")
plt.show()

마지막에 100은 없는 라벨인데 왜 뜨는지..알수없음..<br>
확실히 전체 구매 횟수가 늘어날수록 그만큼 이용한 사람의 수는 눈에 띄게 줄어든다.
애초에 네 번에서 백 번사이 구매한 사람들의 정보를 줬다고 하는데 왜 만든 데이터프레임에 100은 안뜨는걸까,,그래프엔 나오는게 더 웃겨..ㅠㅠ

In [ ]:
# sns.distplot(df_result["count"], hist=False)
# plt.show()

sns.set(font_scale=1.0)
plt.figure(figsize=(15,8)) # 위치가 sns.plot 위에 있어야만 작동하나..?
sns.countplot(x="days_since_prior_order", data=orders)
plt.show()

7.0 과 30.0의 수가 가장 많은 것으로 보아 매 주 또는 매 달 구매가 이루어 질 확률이 가장 높고<br>
일주일 이내에 구매를 다시 진행하는 경우에 비해 일주일~한 달 사이에 다시 구매하는 경우가 눈에 띄게 적다

In [ ]:
sns.countplot(x="order_hour_of_day", data=orders)
plt.show()

하루 중 구매가 이루어지는 시각<br>
아침 7시 이후 주문이 급증하여 9~16시 사이 꾸준히 주문이 많고<br>
16시 이후 주문 수가 감소하여 0~6시 사이의 구매는 거의 없다시피 하다.

In [ ]:
# print(len(train_copy))
# print(train_copy['reordered'][1:2]==0)
# (train_copy['reordered'][0:1]==0).bool()==True
# train_copy.loc[1:2]

In [ ]:
train_copy = train

In [ ]:
train_copy = train_copy.sort_values(['reordered'], ascending=[True])

In [ ]:
tr_re = train_copy[int(555793):]  #tr_re = reordered products in train set
tr_re.head()

In [ ]:
tr_nore = train_copy[:int(555793)]
tr_nore.tail()

In [ ]:
from collections import Counter as cc

pro_name = cc(tr_re['product_id'])
print(pro_name.most_common()[:10])

In [ ]:
pro_name2 = cc(tr_nore['product_id'])
print(pro_name2.most_common()[:10])

In [ ]:
pro_name3 = cc(train['product_id'])
print(pro_name3.most_common()[:10])

In [ ]:
prior_copy = prior

In [ ]:
prior_copy = prior_copy.sort_values(['reordered'], ascending=[True])

In [ ]:
# pr_re = prior_copy  #pr_re = reordered products in prior set
# prior_copy[127000:127100]
# print(prior_copy[16434489:17000000])

In [ ]:
order_product_count = prior.groupby('order_id').count()[['product_id']]
order_product_count.columns = ['product_count']

In [ ]:
orders2 = orders.merge(order_product_count, left_on='order_id', right_index=True)

In [ ]:
index_day = "Sun Mon Tue Wen Thu Fri Sat".split()

In [ ]:
def drawWeekHour(ds, values,  aggfunc=len, title=None, figsize=(18,5) , cmap=None):
    weekhour_ds = ds.pivot_table(index='order_dow', columns='order_hour_of_day', values=values, aggfunc=aggfunc).fillna(0)
    weekhour_ds.index =  [index_day[index] for index in weekhour_ds.index]
    
    plt.figure(figsize=figsize)
    f = sns.heatmap(weekhour_ds, annot=True, fmt="1.1f", linewidths=.5, cmap=cmap) 
    plt.xlabel("Hour")
    plt.ylabel("Day of Week")
    if title:
        plt.title(title, fontsize=15)

In [ ]:
# 시간과 요일간에 재주문이 걸리는 시간에 관한 상관관계
drawWeekHour(orders, values='days_since_prior_order',aggfunc=lambda x: np.mean(x), title="prior orders", cmap='YlGn')

In [ ]:
# 주문 횟수가 많아질 수록 reorder까지의 날짜가 짧아진다.
sns.set(style="whitegrid", palette="colorblind", font_scale=1.5)

orders2.groupby('order_number').agg({'days_since_prior_order':np.mean, 'product_count':np.mean})\
    .plot(figsize=(16,6), title="order_number, prior_order", marker='o' )
plt.ylabel('days since prior order ')
plt.tight_layout()
plt.show()